In [ ]:
import requests
import pandas as pd
import time

In [ ]:
# This api key will change. Update every 24 hours. 

api_key = "RGAPI-c1e1454a-d584-46fe-a337-2ecf95df27bd"

My: {
    "puuid": "Ucia41-iGooJ_05XmU32x3kHTSvTzU5kc9lgzXdPYx1mr1REVuK_kIynCbn2eYhZZkdWbUgZHCdsVA",
    "gameName": "Bigbobo",
    "tagLine": "8249"
}

Ben: {
    "puuid": "EYQmR6wL_lCyXJHLd7fGtOOM21ZpJt8l4Hguodx--vLyQJ3XZsLrgk2dG9gX7b3iKH30qzd1SYFPfA",
    "gameName": "Beniojinini",
    "tagLine": "6042"
}

### Getting PUUID for future use.

In [ ]:
#URL from puuid
#Mine
URL = "https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-puuid/Ucia41-iGooJ_05XmU32x3kHTSvTzU5kc9lgzXdPYx1mr1REVuK_kIynCbn2eYhZZkdWbUgZHCdsVA"

#Ben
#URL = "https://americas.api.riotgames.com/riot/account/v1/accounts/by-puuid/EYQmR6wL_lCyXJHLd7fGtOOM21ZpJt8l4Hguodx--vLyQJ3XZsLrgk2dG9gX7b3iKH30qzd1SYFPfA"

In [ ]:
api_url = URL+"?api_key="+api_key
print(api_url)

In [ ]:
respon= requests.get(api_url)


In [ ]:
player_info = respon.json()
print(player_info)

In [ ]:
player_puuid = player_info["puuid"]
player_puuid

### Matches -- Using PUUID to get my match id

In [ ]:
def match_url(start):
    

    #Mine
    api_match =( 
                "https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/Ucia41-iGooJ_05XmU32x3kHTSvTzU5kc9lgzXdPYx1mr1REVuK_kIynCbn2eYhZZkdWbUgZHCdsVA/ids?start="
                +str(100*start)
                +"&count=100"
                )
    #Ben
    #api_match = "https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/EYQmR6wL_lCyXJHLd7fGtOOM21ZpJt8l4Hguodx--vLyQJ3XZsLrgk2dG9gX7b3iKH30qzd1SYFPfA/ids?start=0&count=100"
    
    return api_match

#How many hundred games I want to get
N=3

def match_list(api_match):
    api_match = api_match+"&api_key="+api_key
    respon = requests.get(api_match)
    matches = respon.json()
    return (matches)
    
matches=[]    
for i in range(0, N):
    url = match_url(i)
    match_chunk = match_list(url)

    matches.extend(match_chunk)  # Extend the list with the results


In [ ]:
def url_list(x):
    
    return  "https://americas.api.riotgames.com/lol/match/v5/matches/"+x+"?api_key="+api_key

# Use a lambda function with map to create the list of URLs
match_url_list = list(map(lambda r: url_list(r), matches))

### Look up stats using matchID -- Using one game only

In [ ]:
respon = requests.get(match_url_list[0])

In [ ]:
stats = respon.json()
#print(stats)

In [ ]:
print(stats.keys())
player_puuid = player_info["puuid"]
print(player_puuid)


print(stats["metadata"])

In [ ]:
stats["info"]

In [ ]:
pd.options.display.max_columns = None
stats["info"].keys()


In [ ]:
### Community Dragon for item name.

In [ ]:
item ="https://raw.communitydragon.org/latest/plugins/rcp-be-lol-game-data/global/default/v1/items.json"
item_json = requests.get(item).json()
item_json


In [ ]:
def json_extract(obj, key):
    arr = []

    def extract(obj, arr, key):
        if isinstance(obj, dict):
            #print(obj.items()) 
            for k, v in obj.items(): #obj.items() returns all key-value pairs of the dictionary obj. 
                if k == key:
                    arr.append(v)

        elif isinstance(obj, list):
            for item in obj:
                extract(item, arr, key)
        return arr

    values = extract(obj, arr, key) #Returning array.
    return values

In [ ]:
item_id = json_extract(item_json, "id")
item_name = json_extract(item_json, "name")

In [ ]:
#map(function, *iterables):
item_dict = dict(map(lambda a, b: (int(a),b),item_id,item_name  ))


In [ ]:
#print(item_name)

In [ ]:

gear_df = pd.DataFrame(item_name,columns=["item_name"])
gear_df

In [ ]:
gear_df.to_excel('gear_list_excel.xlsx', index=False)

### Look up stats using matchID -- Using all games

In [ ]:

def games_combine(x, game_number):
    
    while True:
        
        response = requests.get(x)
        if response.status_code ==429:
            print(game_number)
            print("sleeping")
            time.sleep(30)
            continue
        if response.status_code == 200:
            break
    game_stats = response.json()
    DF_all = pd.DataFrame(game_stats["info"]["participants"])
    DF_all["game_number"] = game_number  # Add a new column to mark the game number
    
    if 'puuid' in DF_all.columns:
        Teamnumber = DF_all.loc[DF_all["puuid"] == player_puuid, "teamId"].iloc[0]
        DF_all["myteam_number"] = Teamnumber
    else:
        print("Warning: 'puuid' column not found.")
        DF_all["myteam_number"] = None  # Set a default value if 'puuid' is missing

    print(game_number)
    
    return DF_all

# Generate the combined DataFrame
DF_all = pd.concat(
    [games_combine(urls, game_number) for game_number, urls in enumerate(match_url_list, start=0)],
    ignore_index=True
)

In [ ]:
DF_all

In [ ]:

#Change item from float to actual name.
DF_all[["item0","item1","item2","item3","item4","item5","item6"]]=DF_all[["item0","item1","item2","item3","item4","item5","item6"]].astype(int)
DF_all["item1"].info()

DF_all = DF_all.replace(item_dict)

### Adding Teamates recognition.

In [ ]:
def teamate_recog(myteam, team):
    try:
        if myteam == team:
            return 1
        else:
            return 0
    except Exception as e:
        return 0  # Default value for other errors

# Apply the function row-wise
DF_all["Teamates"] = DF_all.apply(lambda r: teamate_recog(r["myteam_number"], r["teamId"]), axis=1)

In [ ]:
DF_all

In [ ]:
DF_all.to_excel('my'+str(100*N)+'games.xlsx', index=False)